In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD, Adagrad
from tensorflow.keras.layers import BatchNormalization, Lambda

sys.path.append('..')

import mim.extractors.ab_json as ab_json
import mim.extractors.esc_trop as esc_trop
import mim.massage.esc_trop as mesc_trop
import mim.models.ab_nn as ab_nn
import mim.model_wrapper as model_wrapper
from mim.experiments.ab_glucose import ABGlucose
from mim.extractors.extractor import Data, Container, ECGData, Extractor
from mim.config import GLUCOSE_ROOT
from mim.presenter import Presenter

In [2]:
extractor_kwargs={
    "features": {
        'ecg_mode': 'beat',
        'ecgs': ['index'],
        'features': ['alias','age', 'sex']
    },
    "index": {}
}
extractor = esc_trop.EscTrop(**extractor_kwargs)
data = extractor.get_data()

In [3]:
data['x']["features"][:10]

array([['{74707EA3-AE3C-4925-A80F-1D0395830A82}', 36, 1],
       ['{76D32F25-6B4C-4367-AFBE-47AC6C1C27F2}', 45, 1],
       ['{31603ACF-E64C-4F38-8C37-5CD30974E3A9}', 51, 0],
       ['{1B0B3CE8-4215-4C99-AD4C-D7B9C43BF02E}', 100, 0],
       ['{D7FAC4E9-AA8C-4E3C-B2B7-E3562417174C}', 61, 0],
       ['{74AC3A61-4AE9-4E40-ADCF-B7787AFFA36E}', 85, 1],
       ['{847236EB-C6BA-4F12-A499-DB60DBB0BDC5}', 76, 0],
       ['{3228D6C2-9119-4E4D-9A93-138A8FD5D72A}', 98, 0],
       ['{C866ECC1-F425-4423-B035-B85847AAEC6A}', 70, 1],
       ['{3D9E7229-A7C0-4D1A-B29F-588AC0CF38D6}', 56, 1]], dtype=object)

In [4]:
alias_to_drop = [a[0] for a in data['x']["features"]]

In [5]:
ecg = mesc_trop.make_ecg_table().rename_axis('ecg_id').reset_index().set_index('patient_id')

In [6]:
len(ecg)

486177

In [7]:
ecg

,ecg_id,ecg_date
patient_id,,
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},0,2018-05-03 11:06:09
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},1,2018-05-03 11:07:05
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},2,2018-05-29 13:53:02
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},3,2018-05-29 13:54:40
{000FBD22-3E32-4B73-B37B-C8E3475379FB},4,2017-09-30 10:29:39
...,...,...
{FFFACA99-6D7D-459A-A693-7693FD7A5AFB},536129,2018-06-19 15:17:05
{FFFACA99-6D7D-459A-A693-7693FD7A5AFB},536130,2018-06-20 13:36:47
{FFFACA99-6D7D-459A-A693-7693FD7A5AFB},536131,2018-09-17 08:20:24


In [8]:
ecg = ecg.drop(alias_to_drop)

In [9]:
ecg

,ecg_id,ecg_date
patient_id,,
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},0,2018-05-03 11:06:09
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},1,2018-05-03 11:07:05
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},2,2018-05-29 13:53:02
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},3,2018-05-29 13:54:40
{000FBD22-3E32-4B73-B37B-C8E3475379FB},4,2017-09-30 10:29:39
...,...,...
{FFF8F162-9C49-458B-A980-2D13930A9A95},535929,2019-02-04 08:14:28
{FFF8F162-9C49-458B-A980-2D13930A9A95},535930,2019-02-04 08:42:55
{FFF8F162-9C49-458B-A980-2D13930A9A95},535931,2019-02-05 11:05:03


In [10]:
ed_path = '/mnt/air-crypt/air-crypt-raw/andersb/data/ESC_Trop_' \
          '17-18-2020-09-21/data/ESC_TROP_Vårdkontakt_' \
          'InkluderadeIndexBesök_2017_2018.csv'

col_map = {
    'KontaktId': 'id',
    'Alias': 'alias',
    'Vardkontakt_InskrivningDatum': 'admission_date',
    'Kön': 'sex',
    'Ålder vid inklusion': 'age',
    'MACE inom 30 dagar': 'mace_30_days'
}

ed = pd.read_csv(ed_path, encoding='latin1', sep='|')
ed = ed.loc[:, list(col_map)].rename(columns=col_map)
ed.admission_date = pd.to_datetime(
    ed.admission_date, format="%Y-%m-%d %H:%M:%S.%f")
ed = (
    ed.sort_values(by='admission_date')
    .drop_duplicates(subset=['alias'], keep='last')
    .set_index('alias')
)
ed["birthyear"] = pd.DatetimeIndex(ed["admission_date"]).year-ed["age"]
ed = ed.drop(["id", "admission_date", "age", "mace_30_days"], axis=1)

In [11]:
ed

,sex,birthyear
alias,,
{B2122C7D-35F2-4D8A-A9D4-EF709EF6E92C},M,1930
{74707EA3-AE3C-4925-A80F-1D0395830A82},M,1981
{D095F9B9-45C4-4B3F-975F-AA80AEEDD1FD},F,1982
{76D32F25-6B4C-4367-AFBE-47AC6C1C27F2},M,1972
{31603ACF-E64C-4F38-8C37-5CD30974E3A9},F,1966
...,...,...
{2ECB0472-0A6C-4C6D-BC5F-FBB265D86DBE},F,1915
{F738B7B7-9718-422E-B27F-A8AEAEB8C561},F,1963
{85559731-8D42-4282-919D-9BFB00E0D68B},F,1989


In [12]:
combined = ecg.join(ed,how="left")
combined["age"] = pd.DatetimeIndex(combined["ecg_date"]).year - combined["birthyear"]

In [13]:
combined.sex.isnull().sum()

39251

In [14]:
combined  # Sort on ecg_id?

,ecg_id,ecg_date,sex,birthyear,age
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},0,2018-05-03 11:06:09,M,1982.0,36.0
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},1,2018-05-03 11:07:05,M,1982.0,36.0
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},2,2018-05-29 13:53:02,M,1982.0,36.0
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},3,2018-05-29 13:54:40,M,1982.0,36.0
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},339260,2018-05-03 14:14:46,M,1982.0,36.0
...,...,...,...,...,...
{FFF8F162-9C49-458B-A980-2D13930A9A95},535930,2019-02-04 08:42:55,F,1938.0,81.0
{FFF8F162-9C49-458B-A980-2D13930A9A95},535931,2019-02-05 11:05:03,F,1938.0,81.0
{FFF8F162-9C49-458B-A980-2D13930A9A95},535932,2019-02-14 10:42:38,F,1938.0,81.0
{FFF8F162-9C49-458B-A980-2D13930A9A95},535933,2019-03-05 11:26:38,F,1938.0,81.0


In [15]:
combined = combined.dropna()

In [16]:
combined

,ecg_id,ecg_date,sex,birthyear,age
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},0,2018-05-03 11:06:09,M,1982.0,36.0
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},1,2018-05-03 11:07:05,M,1982.0,36.0
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},2,2018-05-29 13:53:02,M,1982.0,36.0
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},3,2018-05-29 13:54:40,M,1982.0,36.0
{0000C50B-0BB5-489B-89E6-4F3CCFFB3C73},339260,2018-05-03 14:14:46,M,1982.0,36.0
...,...,...,...,...,...
{FFF8F162-9C49-458B-A980-2D13930A9A95},535930,2019-02-04 08:42:55,F,1938.0,81.0
{FFF8F162-9C49-458B-A980-2D13930A9A95},535931,2019-02-05 11:05:03,F,1938.0,81.0
{FFF8F162-9C49-458B-A980-2D13930A9A95},535932,2019-02-14 10:42:38,F,1938.0,81.0
{FFF8F162-9C49-458B-A980-2D13930A9A95},535933,2019-03-05 11:26:38,F,1938.0,81.0


In [20]:
ecg_path = '/mnt/air-crypt/air-crypt-esc-trop/axel/ecg.hdf5'
mode = "raw"
idx = combined.ecg_id.astype(int).values
#np.random.shuffle(idx)
x_train = ECGData(
                ecg_path,
                mode=mode,
                index=idx,
                fits_in_memory=False
            )
y_age = list(combined.age)
y_gender = list(combined.sex.apply(lambda x: 1 if x == 'M' else 0))
data = Container({
    'x': Container({"ecg": x_train}),
    'y': Data(y_gender)
})

In [21]:
print("hello")

hello


In [22]:
model = ab_nn.dyn_cnn(
    train=data,
    conv_dropout=[0.0, 0.3, 0.0],
    conv_filters=[64, 16, 8],
    conv_kernel_size=[64, 16, 16],
    conv_pool_size=[32, 4, 8],
    conv_weight_decay=[0.0001, 0.001, 0.0001],
    conv_final_dense_neurons=10,
    final_dense_neurons=10,
    final_dense_dropout=0.5,
    activation="relu",    
    skip_basic=True,
    ecg_normalization_layer=BatchNormalization()
)
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [23]:
batch_size = 128
dataset = model_wrapper.prepare_dataset(data, batch_size=batch_size)
model.fit(dataset, epochs=20, batch_size=batch_size)

Epoch 1/20
1128/1128 [==============================] - 324s 287ms/step - loss: 0.6931 - accuracy: 0.5220
Epoch 2/20
1128/1128 [==============================] - 315s 279ms/step - loss: 0.6920 - accuracy: 0.5255
Epoch 3/20
1128/1128 [==============================] - 310s 275ms/step - loss: 0.6920 - accuracy: 0.5255
Epoch 4/20
1128/1128 [==============================] - 309s 274ms/step - loss: 0.6920 - accuracy: 0.5255
Epoch 5/20
1128/1128 [==============================] - 313s 278ms/step - loss: 0.6920 - accuracy: 0.5255
Epoch 6/20
1128/1128 [==============================] - 312s 277ms/step - loss: 0.6920 - accuracy: 0.5255
Epoch 7/20
1128/1128 [==============================] - 309s 274ms/step - loss: 0.6920 - accuracy: 0.5255
Epoch 8/20
1128/1128 [==============================] - 315s 279ms/step - loss: 0.6920 - accuracy: 0.5255
Epoch 9/20
1128/1128 [==============================] - 316s 280ms/step - loss: 0.6920 - accuracy: 0.5255
Epoch 10/20
1128/1128 [=======================

In [29]:
np.mean(y_gender)

0.5254527316309423

In [30]:
m = 4.057771e-05
s = 0.0001882498
normalization_layer = Lambda(lambda v: (v - m) / s)

model = ab_nn.dyn_cnn(
    train=data,
    conv_dropout=[0.0, 0.3, 0.0],
    conv_filters=[64, 16, 8],
    conv_kernel_size=[64, 16, 16],
    conv_pool_size=[32, 4, 8],
    conv_weight_decay=[0.0001, 0.001, 0.0001],
    conv_final_dense_neurons=10,
    final_dense_neurons=10,
    final_dense_dropout=0.5,
    activation="relu",    
    skip_basic=True,
    ecg_normalization_layer=normalization_layer
)
optimizer = Adam(learning_rate=0.0003)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
batch_size = 128
dataset = model_wrapper.prepare_dataset(data, batch_size=batch_size)
history = model.fit(dataset, epochs=100, batch_size=batch_size)

Epoch 1/100
1128/1128 [==============================] - 281s 249ms/step - loss: 0.6824 - accuracy: 0.5962
Epoch 2/100
1128/1128 [==============================] - 275s 244ms/step - loss: 0.6003 - accuracy: 0.6956
Epoch 3/100
1128/1128 [==============================] - 272s 241ms/step - loss: 0.5701 - accuracy: 0.7255
Epoch 4/100
1128/1128 [==============================] - 280s 248ms/step - loss: 0.5545 - accuracy: 0.7364
Epoch 5/100
1128/1128 [==============================] - 277s 246ms/step - loss: 0.5449 - accuracy: 0.7446
Epoch 6/100
1128/1128 [==============================] - 275s 244ms/step - loss: 0.5381 - accuracy: 0.7486
Epoch 7/100
1128/1128 [==============================] - 277s 246ms/step - loss: 0.5304 - accuracy: 0.7542
Epoch 8/100
1128/1128 [==============================] - 278s 247ms/step - loss: 0.5262 - accuracy: 0.7559
Epoch 9/100
1128/1128 [==============================] - 280s 249ms/step - loss: 0.5211 - accuracy: 0.7605
Epoch 10/100
1128/1128 [=============

Epoch 77/100
1128/1128 [==============================] - 270s 239ms/step - loss: 0.4407 - accuracy: 0.8160
Epoch 78/100
1128/1128 [==============================] - 275s 244ms/step - loss: 0.4436 - accuracy: 0.8160
Epoch 79/100
1128/1128 [==============================] - 273s 242ms/step - loss: 0.4422 - accuracy: 0.8163
Epoch 80/100
1128/1128 [==============================] - 273s 242ms/step - loss: 0.4389 - accuracy: 0.8185
Epoch 81/100
1128/1128 [==============================] - 275s 244ms/step - loss: 0.4393 - accuracy: 0.8176
Epoch 82/100
1128/1128 [==============================] - 273s 242ms/step - loss: 0.4404 - accuracy: 0.8169
Epoch 83/100
1128/1128 [==============================] - 285s 252ms/step - loss: 0.4392 - accuracy: 0.8179
Epoch 84/100
1128/1128 [==============================] - 271s 240ms/step - loss: 0.4363 - accuracy: 0.8194
Epoch 85/100
1128/1128 [==============================] - 277s 246ms/step - loss: 0.4377 - accuracy: 0.8179
Epoch 86/100
1128/1128 [====

In [31]:
model = ab_nn.dyn_cnn(
    train=data,
    conv_dropout=[0.0, 0.3, 0.0],
    conv_filters=[64, 16, 8],
    conv_kernel_size=[64, 16, 16],
    conv_pool_size=[32, 4, 8],
    conv_weight_decay=[0.0001, 0.001, 0.0001],
    conv_final_dense_neurons=10,
    final_dense_neurons=10,
    final_dense_dropout=0.5,
    activation="relu",    
    skip_basic=True,
    ecg_normalization_layer=BatchNormalization()
)
optimizer = Adam(learning_rate=0.0003)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
batch_size = 128
dataset = model_wrapper.prepare_dataset(data, batch_size=batch_size)
model.fit(dataset, epochs=100, batch_size=batch_size)

Epoch 1/100
1128/1128 [==============================] - 294s 261ms/step - loss: 0.6758 - accuracy: 0.5770
Epoch 2/100
1128/1128 [==============================] - 288s 255ms/step - loss: 0.6211 - accuracy: 0.6767
Epoch 3/100
1128/1128 [==============================] - 294s 260ms/step - loss: 0.5993 - accuracy: 0.6974
Epoch 4/100
1128/1128 [==============================] - 294s 261ms/step - loss: 0.5853 - accuracy: 0.7103
Epoch 5/100
1128/1128 [==============================] - 289s 256ms/step - loss: 0.5747 - accuracy: 0.7202
Epoch 6/100
1128/1128 [==============================] - 288s 255ms/step - loss: 0.5632 - accuracy: 0.7296
Epoch 7/100
1128/1128 [==============================] - 290s 257ms/step - loss: 0.5541 - accuracy: 0.7370
Epoch 8/100
1128/1128 [==============================] - 284s 252ms/step - loss: 0.5453 - accuracy: 0.7417
Epoch 9/100
1128/1128 [==============================] - 287s 254ms/step - loss: 0.5397 - accuracy: 0.7465
Epoch 10/100
1128/1128 [=============

Epoch 77/100
1128/1128 [==============================] - 279s 248ms/step - loss: 0.4353 - accuracy: 0.8214
Epoch 78/100
1128/1128 [==============================] - 280s 248ms/step - loss: 0.4338 - accuracy: 0.8229
Epoch 79/100
1128/1128 [==============================] - 284s 251ms/step - loss: 0.4322 - accuracy: 0.8240
Epoch 80/100
1128/1128 [==============================] - 276s 245ms/step - loss: 0.4328 - accuracy: 0.8228
Epoch 81/100
1128/1128 [==============================] - 282s 250ms/step - loss: 0.4315 - accuracy: 0.8236
Epoch 82/100
1128/1128 [==============================] - 291s 258ms/step - loss: 0.4310 - accuracy: 0.8249
Epoch 83/100
1128/1128 [==============================] - 292s 259ms/step - loss: 0.4298 - accuracy: 0.8247
Epoch 84/100
1128/1128 [==============================] - 277s 245ms/step - loss: 0.4276 - accuracy: 0.8261
Epoch 85/100
1128/1128 [==============================] - 284s 252ms/step - loss: 0.4275 - accuracy: 0.8258
Epoch 86/100
1128/1128 [====

In [33]:
m = 4.057771e-05
s = 0.0001882498
normalization_layer = Lambda(lambda v: (v - m) / s)

model = ab_nn.get_stanford_cnn(
    data,
    ecg_normalization_layer=normalization_layer
)
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
batch_size = 128
dataset = model_wrapper.prepare_dataset(data, batch_size=batch_size)
model.summary()
history = model.fit(dataset, epochs=400, batch_size=batch_size)

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ecg (InputLayer)                [(None, 10000, 8)]   0                                            
__________________________________________________________________________________________________
lambda_18 (Lambda)              (None, 10000, 8)     0           ecg[0][0]                        
__________________________________________________________________________________________________
conv1d_124 (Conv1D)             (None, 10000, 16)    2064        lambda_18[0][0]                  
__________________________________________________________________________________________________
batch_normalization_128 (BatchN (None, 10000, 16)    64          conv1d_124[0][0]                 
___________________________________________________________________________________________

1128/1128 [==============================] - 503s 446ms/step - loss: 1.1116 - accuracy: 0.5230
Epoch 2/400
1128/1128 [==============================] - 510s 452ms/step - loss: 0.7133 - accuracy: 0.5255
Epoch 3/400
1128/1128 [==============================] - 500s 443ms/step - loss: 0.6953 - accuracy: 0.5255
Epoch 4/400
1128/1128 [==============================] - 499s 442ms/step - loss: 0.6931 - accuracy: 0.5254
Epoch 5/400
1128/1128 [==============================] - 491s 435ms/step - loss: 0.7056 - accuracy: 0.5254
Epoch 6/400
1128/1128 [==============================] - 504s 447ms/step - loss: 0.6949 - accuracy: 0.5255
Epoch 7/400
1128/1128 [==============================] - 500s 443ms/step - loss: 0.6946 - accuracy: 0.5254
Epoch 8/400
1128/1128 [==============================] - 497s 441ms/step - loss: 0.6932 - accuracy: 0.5254
Epoch 9/400
1128/1128 [==============================] - 498s 441ms/step - loss: 0.6991 - accuracy: 0.5255
Epoch 10/400
1128/1128 [=========================

1128/1128 [==============================] - 500s 444ms/step - loss: 0.5590 - accuracy: 0.7290
Epoch 78/400
1128/1128 [==============================] - 501s 444ms/step - loss: 0.5505 - accuracy: 0.7353
Epoch 79/400
1128/1128 [==============================] - 497s 441ms/step - loss: 0.5444 - accuracy: 0.7397
Epoch 80/400
1128/1128 [==============================] - 498s 442ms/step - loss: 0.5395 - accuracy: 0.7443
Epoch 81/400
1128/1128 [==============================] - 496s 440ms/step - loss: 0.5352 - accuracy: 0.7469
Epoch 82/400
1128/1128 [==============================] - 501s 444ms/step - loss: 0.5301 - accuracy: 0.7504
Epoch 83/400
1128/1128 [==============================] - 497s 441ms/step - loss: 0.5257 - accuracy: 0.7532
Epoch 84/400
1128/1128 [==============================] - 499s 442ms/step - loss: 0.5224 - accuracy: 0.7562
Epoch 85/400
1128/1128 [==============================] - 498s 442ms/step - loss: 0.5182 - accuracy: 0.7594
Epoch 86/400
1128/1128 [=================

1128/1128 [==============================] - 919s 815ms/step - loss: 0.4550 - accuracy: 0.8052
Epoch 153/400
1128/1128 [==============================] - 976s 865ms/step - loss: 0.4563 - accuracy: 0.8041
Epoch 154/400
1128/1128 [==============================] - 898s 796ms/step - loss: 0.4573 - accuracy: 0.8029
Epoch 155/400
1128/1128 [==============================] - 503s 446ms/step - loss: 0.4548 - accuracy: 0.8044
Epoch 156/400
1128/1128 [==============================] - 508s 450ms/step - loss: 0.4548 - accuracy: 0.8053
Epoch 157/400
1128/1128 [==============================] - 505s 448ms/step - loss: 0.4542 - accuracy: 0.8053
Epoch 158/400
1128/1128 [==============================] - 501s 444ms/step - loss: 0.4546 - accuracy: 0.8053
Epoch 159/400
1128/1128 [==============================] - 503s 445ms/step - loss: 0.4534 - accuracy: 0.8051
Epoch 160/400
1128/1128 [==============================] - 508s 450ms/step - loss: 0.4549 - accuracy: 0.8054
Epoch 161/400
1128/1128 [========

1128/1128 [==============================] - 504s 446ms/step - loss: 0.4382 - accuracy: 0.8158
Epoch 228/400
1128/1128 [==============================] - 506s 449ms/step - loss: 0.4383 - accuracy: 0.8147
Epoch 229/400
1128/1128 [==============================] - 505s 447ms/step - loss: 0.4389 - accuracy: 0.8152
Epoch 230/400
1128/1128 [==============================] - 504s 447ms/step - loss: 0.4376 - accuracy: 0.8160
Epoch 231/400
1128/1128 [==============================] - 505s 448ms/step - loss: 0.4369 - accuracy: 0.8148
Epoch 232/400
1128/1128 [==============================] - 502s 445ms/step - loss: 0.4387 - accuracy: 0.8154
Epoch 233/400
1128/1128 [==============================] - 501s 444ms/step - loss: 0.4358 - accuracy: 0.8171
Epoch 234/400
1128/1128 [==============================] - 500s 443ms/step - loss: 0.4378 - accuracy: 0.8154
Epoch 235/400
1128/1128 [==============================] - 499s 442ms/step - loss: 0.4396 - accuracy: 0.8151
Epoch 236/400
1128/1128 [========

1128/1128 [==============================] - 505s 448ms/step - loss: 0.4281 - accuracy: 0.8211
Epoch 303/400
1128/1128 [==============================] - 506s 448ms/step - loss: 0.4269 - accuracy: 0.8220
Epoch 304/400
1128/1128 [==============================] - 502s 445ms/step - loss: 0.4311 - accuracy: 0.8202
Epoch 305/400
1128/1128 [==============================] - 507s 449ms/step - loss: 0.4269 - accuracy: 0.8228
Epoch 306/400
1128/1128 [==============================] - 529s 469ms/step - loss: 0.4281 - accuracy: 0.8215
Epoch 307/400
1128/1128 [==============================] - 525s 465ms/step - loss: 0.4276 - accuracy: 0.8222
Epoch 308/400
1128/1128 [==============================] - 525s 465ms/step - loss: 0.4266 - accuracy: 0.8222
Epoch 309/400
1128/1128 [==============================] - 527s 467ms/step - loss: 0.4268 - accuracy: 0.8212
Epoch 310/400
1128/1128 [==============================] - 521s 462ms/step - loss: 0.4263 - accuracy: 0.8236
Epoch 311/400
1128/1128 [========

1128/1128 [==============================] - 522s 463ms/step - loss: 0.4201 - accuracy: 0.8264
Epoch 378/400
1128/1128 [==============================] - 518s 459ms/step - loss: 0.4187 - accuracy: 0.8264
Epoch 379/400
1128/1128 [==============================] - 526s 466ms/step - loss: 0.4171 - accuracy: 0.8276
Epoch 380/400
1128/1128 [==============================] - 528s 468ms/step - loss: 0.4188 - accuracy: 0.8257
Epoch 381/400
1128/1128 [==============================] - 528s 468ms/step - loss: 0.4168 - accuracy: 0.8273
Epoch 382/400
1128/1128 [==============================] - 527s 467ms/step - loss: 0.4175 - accuracy: 0.8266
Epoch 383/400
1128/1128 [==============================] - 527s 467ms/step - loss: 0.4181 - accuracy: 0.8262
Epoch 384/400
1128/1128 [==============================] - 523s 464ms/step - loss: 0.4172 - accuracy: 0.8275
Epoch 385/400
1128/1128 [==============================] - 522s 462ms/step - loss: 0.4176 - accuracy: 0.8271
Epoch 386/400
1128/1128 [========

In [ ]:
p = Presenter("ABGlucose")

In [ ]:
p.describe()

In [ ]:
p.train_test_scores("KERAS_HIDDEN_10")

In [ ]:
list(p.results["KERAS_HIDDEN_10"]["predictions"]["prediction"])